In [1]:
!pip install transformers datasets   #installing modules

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM                     #importing  particular classes from these modules
import torch
# the model name and load the tokenizer and model
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"               #check for cpu or gpu as the run time type
model = model.to(device)

# Print out which device we're using (GPU or CPU)
print(device)

cuda


In [4]:
from datasets import load_dataset     #importing functions from dataset module

# Load the CNN/DailyMail dataset, which contains articles and summaries
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train")

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [5]:
# Split the dataset into training and testing subsets
dataset_split = dataset.train_test_split(test_size=0.1)

# Further reduce the training set size for faster testing during development
small_train_dataset = dataset_split['train'].train_test_split(test_size=0.9775)['train']
eval_dataset = dataset_split['test']
print(dataset[0])

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [6]:
def preprocess_function(examples):
  # Extract the articles from the dataset
  inputs = [doc for doc in examples['article']]

  # Tokenize the articles (inputs) with padding and truncation to a max length of 512
  model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True, return_tensors="pt")

  # Tokenize the summaries (labels) using the target tokenizer context
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['highlights'], max_length=128, padding="max_length", truncation=True, return_tensors="pt")

  # Attach the tokenized summaries as labels to the model inputs
  model_inputs["labels"] = labels["input_ids"]

  # Move the tokenized inputs and labels to the appropriate device (GPU/CPU)
  model_inputs = {k: v.to(device) for k, v in model_inputs.items()}

  return model_inputs

In [7]:
# Tokenize the small training dataset
tokenized_train_dataset = small_train_dataset.map(preprocess_function, batched=True)

# Tokenize the evaluation dataset
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5814 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/28712 [00:00<?, ? examples/s]

In [8]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',              # Directory to save the model checkpoints
    evaluation_strategy="epoch",         # Evaluate the model at the end of every epoch
    learning_rate=2e-5,                  # Learning rate for the optimizer
    per_device_train_batch_size=8,       # Batch size for training
    per_device_eval_batch_size=8,        # Batch size for evaluation
    weight_decay=0.01,                   # Regularization to prevent overfitting
    save_total_limit=3,                  # Only keep the last 3 checkpoints
    num_train_epochs=3,                  # Number of training epochs
    predict_with_generate=True,          # Enable text generation during evaluation
    logging_dir="./logs"                 # Directory for storing training logs
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer = Seq2SeqTrainer(
    model=model,                         # The model to be trained
    args=training_args,                  # The training arguments defined earlier
    train_dataset=tokenized_train_dataset,  # The tokenized training dataset
    eval_dataset=tokenized_eval_dataset,    # The tokenized evaluation dataset
    tokenizer=tokenizer                  # The tokenizer to handle input and output
)

In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,5.643200,1.380887
2,1.695000,1.194710
3,1.326000,1.187952


TrainOutput(global_step=2181, training_loss=2.4099485947417865, metrics={'train_runtime': 2377.6241, 'train_samples_per_second': 7.336, 'train_steps_per_second': 0.917, 'total_flos': 3242302183047168.0, 'train_loss': 2.4099485947417865, 'epoch': 3.0})

In [11]:
# Evaluate the model on the evaluation dataset
metrics = trainer.evaluate()

In [12]:
def summarize(text):
  # Tokenize the input text and move it to the correct device
  inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)

  # Generate the summary using the fine-tuned model
  summary_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4,    temperature=0.6,  # Control creativity
    top_k=50,  # Top-k sampling for diversity
    top_p=0.9,  # Top-p nucleus sampling
    repetition_penalty=2.0,  # Avoid repetition
 early_stopping=True)

  # Decode the generated summary back into text and return it
  return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [13]:
print(summarize(
    """
Person A: Hey, did you hear that we’re getting new time-tracking software next month?

Person B: Really? I hadn’t heard. What’s it called?

Person A: It’s called "TimeMaster." Apparently, it’s going to make tracking hours and project timelines easier, especially with the new hybrid work model.

Person B: Interesting. But isn’t the software we’re using now good enough?

Person A: It works, but TimeMaster has more advanced features. It integrates with the payroll system and automatically generates reports. Plus, managers can approve timesheets with just one click.

Person B: That does sound convenient. Do we need to install it ourselves?

Person A: No, IT will handle the installation. But we’ll have to learn how to use it. There’s a training session scheduled for next Wednesday.

Person B: I’ll mark my calendar for that. Is it going to replace the whole system we use, or will it work alongside it?

Person A: It’s going to replace the old software completely. We’ll migrate all our data over to TimeMaster, so everything will be centralized.

Person B: Sounds like a big change. I hope the transition goes smoothly.

Person A: Yeah, there’s always a bit of a challenge with new systems, but they’ve said they’ll offer support throughout the process.

Person B: Good to know. I guess we’ll just have to give it a shot and see how it works.

Person A: Exactly. Let’s stay positive about it. I think it’s going to save us a lot of hassle once we get used to it.

Person B: I hope so too. Thanks for filling me in!

Person A: No problem. See you at the training!
"""
))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


TimeMaster is going to make tracking hours and project timelines easier, especially with the new hybrid work model. It integrates with the payroll system and automatically generates reports. Managers can approve timesheets with just one click.


In [14]:
model.save_pretrained('./my_summerizer')
tokenizer.save_pretrained('./my_summerizer')

('./my_summerizer/tokenizer_config.json',
 './my_summerizer/special_tokens_map.json',
 './my_summerizer/spiece.model',
 './my_summerizer/added_tokens.json',
 './my_summerizer/tokenizer.json')

In [17]:
from huggingface_hub import notebook_login
notebook_login()

In [19]:
from google.colab import userdata
HF_T= userdata.get('HF_T')

model.push_to_hub("Trizer", token=HF_T)
tokenizer.push_to_hub("Trizer", token=HF_T)


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Abir866/Trizer/commit/36a75b599b7b49b314345c5d47cee4247f48f5b4', commit_message='Upload tokenizer', commit_description='', oid='36a75b599b7b49b314345c5d47cee4247f48f5b4', pr_url=None, pr_revision=None, pr_num=None)